<a href="https://colab.research.google.com/github/Rweiber/Check-News/blob/main/Check_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai
!pip install -q ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.0 MB/s eta 0:00:00


In [5]:
# Importa as bibliotecas necessárias
from pathlib import Path
import hashlib
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from google.colab import userdata

# Configura a API Key do Google Generative AI (obtenha a sua em https://developers.generativeai.google/)
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

# Define as configurações do modelo Gemini Pro
generation_config = {
    "temperature": 0.7, # Controla a criatividade do modelo (valores mais altos = mais criativo)
    "top_p": 0.95, # Limita a probabilidade das próximas palavras
    "top_k": 0, # Não limita o número de palavras consideradas
    "max_output_tokens": 8192, # Define o limite máximo de tokens na resposta
}

# Define as configurações de segurança para o modelo
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT", # Assédio
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de assédio
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH", # Discurso de ódio
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de discurso de ódio
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", # Conteúdo sexualmente explícito
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de conteúdo sexualmente explícito
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT", # Conteúdo perigoso
        "threshold": "BLOCK_MEDIUM_AND_ABOVE", # Bloqueia respostas com nível médio ou alto de conteúdo perigoso
    },
]

# Define a instrução inicial para o modelo
system_instruction = "Haja como um investigador de noticias, que pesquisa nas fontes de noticias mais qualificadas do brasil e retorno uma porcentagem do quão verdadeira é a noticia, tambem preciso que descreva a imagem, por completa no inicio da resposta, no final sempre coloque as fontes de onde pesquisou e o o link da fonte"

# Cria uma instância do modelo Gemini Pro
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    system_instruction=system_instruction,
    safety_settings=safety_settings,
)

# Cria os widgets
upload_button = widgets.FileUpload(accept="image/*", multiple=False, description="Enviar imagem/print para verificação", layout=widgets.Layout(width='70%'))
output_text = widgets.Output()

# Define a função que será chamada quando o usuário carregar uma imagem
def on_upload_change(change):
    if change["new"]:
        # Obtém o nome do arquivo e o conteúdo da imagem carregada
        uploaded_filename = list(upload_button.value.keys())[0]
        uploaded_content = upload_button.value[uploaded_filename]["content"]
        # Cria um arquivo temporário para armazenar o conteúdo da imagem
        image_path = Path(uploaded_filename)
        with open(image_path, "wb") as f:
            f.write(uploaded_content)
        # Executa a análise da imagem
        with output_text:
            print("Analisando a imagem...")
            convo = model.start_chat(history=[
                {
                    "role": "user",
                    "parts": [genai.upload_file(image_path)]
                },
            ])

            # Exemplos de Few-Shot Prompting
            few_shot_examples = """
## Exemplo 1
**Prompt:** A imagem a ser analisada é um print de tela de uma publicação no Facebook. A imagem mostra um texto afirmando que o governo irá distribuir cestas básicas gratuitas para toda a população.
**Resposta:** A imagem mostra um print de tela de uma publicação no Facebook, com a data de 10/10/2023, proveniente de um perfil chamado "Notícias Bombásticas". O texto afirma: "Governo anuncia distribuição de cestas básicas gratuitas para toda a população! Cadastre-se já!".  A probabilidade da notícia ser verdadeira é 10%. A informação não foi confirmada por nenhuma fonte oficial e circula em sites conhecidos por propagação de notícias falsas. Fontes: [Agência Lupa](https://www.lupa.com.br), [Aos Fatos](https://aosfatos.org/).

## Exemplo 2
**Prompt:** A imagem a ser analisada é uma foto de um protesto. A imagem mostra diversas pessoas segurando cartazes com frases contra a corrupção.
**Resposta:** A imagem mostra uma fotografia de um protesto com grande concentração de pessoas. Os cartazes exibem frases como "Basta de Corrupção" e "Fora Corruptos". A probabilidade da notícia ser verdadeira é 90%. Diversos veículos de comunicação reportaram protestos contra a corrupção na data e local indicados na imagem. Fontes: [G1](https://g1.globo.com/), [Folha de S.Paulo](https://www.folha.uol.com.br).
            """

            # Mensagem com Few-Shot Prompting
            response = convo.send_message(f"""
{few_shot_examples}

## Nova Imagem:
A imagem a ser analisada é um print de tela de uma publicação em rede social ou foto tirada de revista ou jornal. Descreva a imagem detalhadamente, incluindo o texto presente, a fonte de publicação, data, e outros elementos visuais relevantes. Pesquise a veracidade da notícia apresentada na imagem utilizando fontes confiáveis como Agência Lupa, Aos Fatos, Estadão Verifica, G1, Folha de S.Paulo, UOL, etc. Inclua também agências internacionais de notícias reconhecidas. Com base nas informações encontradas, determine a probabilidade da notícia ser verdadeira e apresente como uma porcentagem, justificando sua avaliação. Apresente as fontes utilizadas em formato de lista, incluindo o link para cada fonte consultada.
            """)
            display(Markdown(f"---\n**Resposta:** {response.text}"))
        # Remove o arquivo temporário da imagem
        image_path.unlink()

# Configura a função on_upload_change para ser chamada quando o valor do upload_button mudar
upload_button.observe(on_upload_change, names="value")

# Exibe os widgets na tela
display(upload_button)
display(output_text)

FileUpload(value={}, accept='image/*', description='Enviar imagem/print para verificação', layout=Layout(width…

Output()